In [1]:
from heliopy.data import wind
from datetime import datetime
import matplotlib.pyplot as plt
import astropy.constants as const
import astropy.units as u
import numpy as np
import aiapy.calibrate

import solar_library

import importlib

Set up the in-situ intervals
---

In [7]:
intervals = {2: {'dtimes': (datetime(2013, 1, 24), datetime(2013, 1, 25)),
                 'Bcorona': 255 * u.G},
             1: {'dtimes': (datetime(2008, 1, 12, 14), datetime(2008, 1, 13, 12)),
                 'Bcorona': 19.2 * u.G}}

Calculate derived parameters
----

In [11]:
for i in [1, 2]:
    interval = intervals[i]
    # Load data
    wind_data = wind.swe_h1(*interval['dtimes'])
    
    # Calculate derived parameters
    # Mass flux
    mflux = wind_data.quantity('Proton_Np_nonlin') * -wind_data.quantity('Proton_VX_nonlin') * const.au**2
    wind_data = wind_data.add_column('Mass flux', mflux.to(1 / u.s))
    # Magnetic field strength
    modB = np.sqrt(wind_data.quantity('BX')**2 + wind_data.quantity('BY')**2 + wind_data.quantity('BZ')**2)
    wind_data = wind_data.add_column('|B|', modB)

    # Scale mass flux to coronal base
    Bcorona = interval['Bcorona']
    mflux_corona = (mflux / 1e35 * Bcorona / modB * const.R_sun**2 / const.au**2).to(1/ u.s)
    
    # Save mass flux values
    np.save(f'data/mflux_corona_{i}.npy', mflux_corona.value)